In [1]:
import pandas as pd

# read in

In [2]:
df = pd.read_csv('../../2_Feature_Engineering/export/randomized_balanced_dataset.csv')
df.head()

,raw_text,severity,binary_label
0,"Ich warte schon darauf, dass man die Terrorist...",0.0,False
1,Holt Björn Höcke ins Boot vielleicht haben wir...,0.0,False
2,"Deutscher, turkmenischer Abstammung bitte. Was...",0.0,False
3,"Wer glaubt die gehen wieder , ist einfach nur ...",0.0,False
4,Man wünscht sich eine schlagkräftige Bürgerweh...,0.0,False


# clean

In [4]:
import sys
sys.path.insert(0, r'..\..\0_common')
from model_helpers import clean_all as ci

In [5]:
df.head()

,raw_text,severity,binary_label
0,"Ich warte schon darauf, dass man die Terrorist...",0.0,False
1,Holt Björn Höcke ins Boot vielleicht haben wir...,0.0,False
2,"Deutscher, turkmenischer Abstammung bitte. Was...",0.0,False
3,"Wer glaubt die gehen wieder , ist einfach nur ...",0.0,False
4,Man wünscht sich eine schlagkräftige Bürgerweh...,0.0,False


In [6]:
df['cleaned_text'] = df['raw_text'].apply(ci)

In [7]:
df.head()

,raw_text,severity,binary_label,cleaned_text
0,"Ich warte schon darauf, dass man die Terrorist...",0.0,False,warte schon darauf das terroristen opfern mach...
1,Holt Björn Höcke ins Boot vielleicht haben wir...,0.0,False,holt björn höcke boot vielleicht letzte chance...
2,"Deutscher, turkmenischer Abstammung bitte. Was...",0.0,False,deutscher turkmenischer abstammung bitte graue...
3,"Wer glaubt die gehen wieder , ist einfach nur ...",0.0,False,wer glaubt gehen einfach blauäugig
4,Man wünscht sich eine schlagkräftige Bürgerweh...,0.0,False,wünscht schlagkräftige bürgerwehr sophienhof p...


# vectorize

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
tfidf_vec = TfidfVectorizer(ngram_range=(2,5), analyzer='char', stop_words='german', max_features=10000)
tfidf_wb_vec = TfidfVectorizer(ngram_range=(2,5), analyzer='char_wb', stop_words='german', max_features=10000)
count_vec = CountVectorizer(ngram_range=(2,5), analyzer='char', stop_words='german', max_features=10000)
count_wb_vec = CountVectorizer(ngram_range=(2,5), analyzer='char_wb', stop_words='german', max_features=10000)

In [10]:
tfidf_features = tfidf_vec.fit_transform(df['cleaned_text'])
tfidf_wb_features = tfidf_wb_vec.fit_transform(df['cleaned_text'])
count_features = count_vec.fit_transform(df['cleaned_text'])
count_wb_features = count_wb_vec.fit_transform(df['cleaned_text'])

In [11]:
tfidf_features.shape

(1828, 10000)

In [12]:
tfidf_wb_features.shape

(1828, 10000)

In [13]:
count_features.shape

(1828, 10000)

In [14]:
count_wb_features.shape

(1828, 10000)

# main function

In [15]:
def split_train_score(features, labels, clf):
    
    # split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)
    
    #train    
    clf.fit(X_train, y_train)
    
    # score
    from sklearn.metrics import precision_recall_fscore_support as score
    y_pred = lr.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label=True, average='binary') # pos_label= wenn nicht binary, dann label angeben
    
    print('Precision: {} / Recall: {} / Accuracy: {} / Fscore: {}'.format(round(precision, 3), 
                                                         round(recall, 3), 
                                                         round((y_pred==y_test).sum() / len(y_pred)),
                                                         round(fscore, 3)))
    # confusion matrix
    from sklearn.metrics import confusion_matrix
    matrix = confusion_matrix(y_test, y_pred)
    m_df = pd.DataFrame(
    matrix, 
    columns=["Negatives", "Positives"],
    index=["Negatives", "Positives"])
    print(m_df)
    

In [16]:
options = {'TF-IDF': tfidf_features, 'TF-IDF_with_wb': tfidf_wb_features, 'CountVectorizer': count_features, 'CountVectorizer_with_wb': count_wb_features}

# logistic regression

In [17]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

for index, option in options.items():
    print(index)
    split_train_score(option, df['binary_label'], lr)
    split_train_score(option, df['binary_label'], lr)
    split_train_score(option, df['binary_label'], lr)
    print()

TF-IDF
Precision: 0.822 / Recall: 0.552 / Accuracy: 1.0 / Fscore: 0.66
           Negatives  Positives
Negatives        267         30
Positives        113        139
Precision: 0.91 / Recall: 0.5 / Accuracy: 1.0 / Fscore: 0.645
           Negatives  Positives
Negatives        253         14
Positives        141        141
Precision: 0.84 / Recall: 0.581 / Accuracy: 1.0 / Fscore: 0.687
           Negatives  Positives
Negatives        268         28
Positives        106        147

TF-IDF_with_wb
Precision: 0.829 / Recall: 0.549 / Accuracy: 1.0 / Fscore: 0.66
           Negatives  Positives
Negatives        263         29
Positives        116        141
Precision: 0.832 / Recall: 0.605 / Accuracy: 1.0 / Fscore: 0.701

C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



           Negatives  Positives
Negatives        256         32
Positives        103        158
Precision: 0.841 / Recall: 0.54 / Accuracy: 1.0 / Fscore: 0.657
           Negatives  Positives
Negatives        257         27
Positives        122        143

CountVectorizer
Precision: 0.779 / Recall: 0.69 / Accuracy: 1.0 / Fscore: 0.732
           Negatives  Positives
Negatives        244         50
Positives         79        176
Precision: 0.76 / Recall: 0.736 / Accuracy: 1.0 / Fscore: 0.748
           Negatives  Positives
Negatives        236         59
Positives         67        187
Precision: 0.752 / Recall: 0.71 / Accuracy: 1.0 / Fscore: 0.731
           Negatives  Positives
Negatives        238         59
Positives         73        179

CountVectorizer_with_wb
Precision: 0.705 / Recall: 0.719 / Accuracy: 1.0 / Fscore: 0.712
           Negatives  Positives
Negatives        220         76
Positives         71        182
Precision: 0.705 / Recall: 0.733 / Accuracy: 1.0 / Fscore: 0

# Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

for index, option in options.items():
    print(index)
    split_train_score(option, df['binary_label'], rf)
    split_train_score(option, df['binary_label'], rf)
    split_train_score(option, df['binary_label'], rf)
    print()

TF-IDF
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        316          0
Positives        233          0


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        298          0
Positives        251          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        291          0
Positives        258          0

TF-IDF_with_wb


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        298          0
Positives        251          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        320          0
Positives        229          0


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        312          0
Positives        237          0

CountVectorizer
Precision: 0.388 / Recall: 0.275 / Accuracy: 1.0 / Fscore: 0.322
           Negatives  Positives
Negatives        215        101
Positives        169         64
Precision: 0.495 / Recall: 0.357 / Accuracy: 1.0 / Fscore: 0.415
           Negatives  Positives
Negatives        190         96
Positives        169         94
Precision: 0.491 / Recall: 0.329 / Accuracy: 1.0 / Fscore: 0.394
           Negatives  Positives
Negatives        215         85
Positives        167         82

CountVectorizer_with_wb
Precision: 0.918 / Recall: 0.922 / Accuracy: 1.0 / Fscore: 0.92
           Negatives  Positives
Negatives        286         20
Positives         19        224
Precision: 0.887 / Recall: 0.895 / Accuracy: 1.0 / Fscore: 0.891
           Negatives  Positives
Negatives        285         27
Positives         25        212

# Multinomial Naive Bayes

In [19]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

for index, option in options.items():
    print(index)
    split_train_score(option, df['binary_label'], mnb)
    split_train_score(option, df['binary_label'], mnb)
    split_train_score(option, df['binary_label'], mnb)
    print()

TF-IDF
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        287          0
Positives        262          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        296          0
Positives        253          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        296          0
Positives        253          0

TF-IDF_with_wb
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        319          0
Positives        230          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        275          0
Positives        274          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        300          0
Positives        249          0

CountVectorizer
Precision: 0.483 / Recall: 0.3

C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\li

# Support Vector Machine

In [20]:
from sklearn.linear_model import SGDClassifier
svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None) # params only copied

for index, option in options.items():
    print(index)
    split_train_score(option, df['binary_label'], svm)
    split_train_score(option, df['binary_label'], svm)
    split_train_score(option, df['binary_label'], svm)
    print()

TF-IDF


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        308          0
Positives        241          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        293          0
Positives        256          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        297          0
Positives        252          0

TF-IDF_with_wb
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        310          0
Positives        239          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        308          0
Positives        241          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        313          0
Positives        236          0

CountVectorizer
Precision: 0.489 / Recall: 0.362 / Ac

--> option 'char_wb' on count vectorizer seems to improve the results significantly on almost every algorithm

# compare cv with char_wb

In [21]:
def split_train_score_compare(features, labels, clf):
    
    # split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)
    
    #train    
    clf.fit(X_train, y_train)
    
    # score
    from sklearn.metrics import precision_recall_fscore_support as score
    y_pred = lr.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label=True, average='binary') # pos_label= wenn nicht binary, dann label angeben
    return round(precision, 3), round(recall, 3), round((y_pred==y_test).sum() / len(y_pred)), round(fscore, 3)

In [22]:
scores = {}

In [23]:
winner_algo = []
for i in range(20):
    scores['lr'] = split_train_score_compare(count_wb_features, df['binary_label'], lr)
    scores['rf'] = split_train_score_compare(count_wb_features, df['binary_label'], rf)
    scores['mnb'] = split_train_score_compare(count_wb_features, df['binary_label'], mnb)
    scores['svm'] = split_train_score_compare(count_wb_features, df['binary_label'], svm)
    
    score_df = pd.DataFrame(list(scores.values()), index=list(scores.keys()), columns=['precision', 'recall', 'accuracy', 'fscore']).reset_index().sort_values('fscore', ascending=False)
    winner_algo.append((score_df.iloc[0]['index'], score_df.iloc[0]['fscore'], score_df.iloc[0]['accuracy']))
    print(score_df)
winner_algo

C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


  index  precision  recall  accuracy  fscore
1    rf      0.951   0.886       1.0   0.917
3   svm      0.931   0.871       1.0   0.900
2   mnb      0.914   0.883       1.0   0.898
0    lr      0.785   0.651       1.0   0.711


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


  index  precision  recall  accuracy  fscore
3   svm      0.945   0.932       1.0   0.938
1    rf      0.943   0.909       1.0   0.926
2   mnb      0.930   0.911       1.0   0.920
0    lr      0.748   0.715       1.0   0.731


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

  index  precision  recall  accuracy  fscore
2   mnb      0.957   0.907       1.0   0.932
1    rf      0.909   0.916       1.0   0.913
3   svm      0.920   0.885       1.0   0.902
0    lr      0.733   0.691       1.0   0.711


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


  index  precision  recall  accuracy  fscore
3   svm      0.901   0.917       1.0   0.909
1    rf      0.923   0.890       1.0   0.906
2   mnb      0.908   0.901       1.0   0.904
0    lr      0.737   0.696       1.0   0.716
  index  precision  recall  accuracy  fscore
2   mnb      0.940   0.916       1.0   0.928
1    rf      0.933   0.902       1.0   0.917
3   svm      0.930   0.897       1.0   0.913
0    lr      0.752   0.690       1.0   0.720


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[('rf', 0.917, 1.0),
 ('svm', 0.938, 1.0),
 ('rf', 0.923, 1.0),
 ('rf', 0.919, 1.0),
 ('rf', 0.929, 1.0),
 ('svm', 0.933, 1.0),
 ('svm', 0.932, 1.0),
 ('rf', 0.918, 1.0),
 ('svm', 0.913, 1.0),
 ('svm', 0.912, 1.0),
 ('svm', 0.925, 1.0),
 ('mnb', 0.911, 1.0),
 ('mnb', 0.928, 1.0),
 ('mnb', 0.929, 1.0),
 ('svm', 0.925, 1.0),
 ('svm', 0.933, 1.0),
 ('rf', 0.934, 1.0),
 ('mnb', 0.932, 1.0),
 ('svm', 0.909, 1.0),
 ('mnb', 0.928, 1.0)]

In [24]:
pd.DataFrame(winner_algo, columns=['algo', 'fscore', 'accuracy']).sort_values(['algo','fscore'], ascending=[True, False])

,algo,fscore,accuracy
17,mnb,0.932,1.0
13,mnb,0.929,1.0
12,mnb,0.928,1.0
19,mnb,0.928,1.0
11,mnb,0.911,1.0
16,rf,0.934,1.0
4,rf,0.929,1.0
2,rf,0.923,1.0
3,rf,0.919,1.0
7,rf,0.918,1.0


# exports

In [25]:
def export_function(clf, clf_name, features, labels):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)

    print(clf_name)
    print('-------')
    
    #train    
    clf.fit(X_train, y_train)

    # score
    from sklearn.metrics import precision_recall_fscore_support as score
    y_pred = lr.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label=True, average='binary') # pos_label= wenn nicht binary, dann label angeben
    print('Precision: {} / Recall: {} / Accuracy: {} / Fscore: {}'.format(round(precision, 3), 
                                                             round(recall, 3), 
                                                             round((y_pred==y_test).sum() / len(y_pred)),
                                                             round(fscore, 3)))
    # confusion matrix
    from sklearn.metrics import confusion_matrix
    matrix = confusion_matrix(y_test, y_pred)
    m_df = pd.DataFrame(
    matrix, 
    columns=["Negatives", "Positives"],
    index=["Negatives", "Positives"])
    print(m_df)
    
    # export
    import pickle
    pickle.dump(clf, open('exports/{}_model.pkl'.format(clf_name), 'wb'))
    
    print()

In [26]:
export_function(svm, 'svm', count_wb_features, df['binary_label'])
export_function(mnb, 'mnb', count_wb_features, df['binary_label'])
export_function(rf, 'rf', count_wb_features, df['binary_label'])

svm
-------
Precision: 0.932 / Recall: 0.893 / Accuracy: 1.0 / Fscore: 0.912
           Negatives  Positives
Negatives        289         16
Positives         26        218

mnb
-------
Precision: 0.935 / Recall: 0.91 / Accuracy: 1.0 / Fscore: 0.922
           Negatives  Positives
Negatives        314         14
Positives         20        201

rf
-------
Precision: 0.936 / Recall: 0.877 / Accuracy: 1.0 / Fscore: 0.906
           Negatives  Positives
Negatives        282         15
Positives         31        221

